In [2]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

# Getting URLs of website, pagination
- Website: https://m.baobeihuijia.com/PhotoShow/list.aspx?tid=1&page=1
- Data scraped on 6 August, 5.42pm, so additional listings will not be included. Total pages: 1494, with 35 listings on each page.

In [ ]:
url_full_list = []
time_start = time.time()
for page_num in range(0,1494): 
# for page_num in range(1,1488): 
    try:
        url= f"https://m.baobeihuijia.com/PhotoShow/list.aspx?tid=1&page={page_num}"
        response = requests.get(url)
        doc = BeautifulSoup(response.text, 'html.parser')
        for link in doc.select('.listBorder a'):
            link_url = link.get('href').strip("../../")
            #print(link_url)
            url_complete = f"https://m.baobeihuijia.com/{link_url}"
            url_full_list.append(url_complete)
    except Exception as e:
        print(e)
        df = pd.DataFrame(url_full_list, columns =['urls'])
        df.to_csv('baobei.csv', index=False)
        raise
    print("up to page", page_num, "we found", len(url_full_list), "urls in total")
time_end = time.time()
print('time taken', time_end - time_start, 's')
#df = pd.DataFrame(url_full_list, columns =['urls'])
#df.to_csv('baobei_p331.csv', index=False)

In [ ]:
df = pd.DataFrame(url_full_list, columns =['urls'])
df.shape

In [ ]:
# finding duplicated rows
df.loc[df.duplicated(), :]

In [ ]:
# get rid of duplicate url, 35 urls are removed, end with 52255 rows
df = df.drop_duplicates()
df.shape

In [ ]:
# saving urls into csv
df.to_csv("final_baobei_urls.csv")

# Processing URLs through BeautifulSoup

In [2]:
#pretend to be a human clicking from browsers by creating faking headers
headers = {
    "User-Agent":"User-Agent: Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Mobile Safari/537.36",
    "Host":"m.baobeihuijia.com",
    "Connection":"keep-alive"
}

In [ ]:
# total urls: 52,255
# create multiple scrapers that scrape 10,000 rows each to speed up process + 
# prevent entire data from breaking if something breaks along the way
# then combine them together
full_list = []
url_no = 0
for link in df['urls'][0:10000]: 
    
    try:
        #extend connection time by timeout
        print("scraping url number:",url_no)
        response = requests.get(link, headers=headers,timeout=40) 
        doc = BeautifulSoup(response.text, 'html.parser')
        metadatum = doc.select('.col-xs-8')
        
    except requests.exceptions.RequestException as e:
        print(e)

    try:
        row = {}
        row['Type'] = metadatum[0].text
        row['ID'] = metadatum[1].text
        row['Name'] = metadatum[2].text
        row['Sex'] = metadatum[3].text
        row['DOB'] = metadatum[4].text
        row['Height_when_missing'] = metadatum[5].text
        row['Missing_date'] = metadatum[6].text
        row['Missing_person_current_location'] = metadatum[7].text
        row['Missing_location'] = metadatum[8].text
        row['Unique_traits'] = metadatum[9].text
        row['Other_info'] = metadatum[10].text
        row['Registration_datetime'] = metadatum[11].text
        full_list.append(row)
        print(url_no,"finished")
        url_no = url_no + 1
    except:
        print("skip url number",url_no)
        url_no = url_no + 1
        continue
    time.sleep(1)

# Geocoding the df

In [3]:
# read in ALL the csvs
df1 = pd.read_csv("baobei0to10000.csv")
df2 = pd.read_csv("baobei_10000_20000.csv")
df3 = pd.read_csv("baobei_20000_30000.csv")
df4 = pd.read_csv("baobei_30000_40000.csv")
df5 = pd.read_csv("baobei40000toend.csv")

In [ ]:
# join them into one df
frames = [df1, df2, df3, df4, df5]
df = pd.concat(frames)

In [ ]:
# save them into csv!
df.to_csv("cleaned_8Aug.csv")

In [ ]:
# Google cloud API key: GOOGLE API KEY
# guide: https://github.com/googlemaps/google-maps-services-python
# check this out: https://github.com/jsoma/NICAR20-geocoding-apis/blob/master/Geocoding%20API%20worksheet-completed.ipynb

In [ ]:
# geocode template; create multiple notebooks to geocode 1,000 each 
# 1K bc sometimes geocoder freezes, so I found 1K to be a sweetspot

import pandas as pd
import googlemaps
from datetime import datetime
!pip install geocoder tqdm==4.42.1
import geocoder
from tqdm import tqdm
tqdm.pandas()
from pandas.io.json import json_normalize

In [ ]:
df = pd.read_csv("cleaned_8Aug.csv")
df1 = df[9000:10000] # specify the rows you wanna scrape

def google_geocode(row):
    # Geocode the address
    address = row['Missing_location']
    g = geocoder.google(address, key='GOOGLE API KEY')

    # Combine the original data with the geocoded data
    combined = {**row, 'geo': g.json}

    return combined

geocoded = df9.progress_apply(google_geocode, axis=1)
geocoded = json_normalize(geocoded, sep='_')

In [ ]:
# i just want the latlong and city
geocoded = geocoded[[
    'ID', 
    'Name', 
    'Sex', 
    'DOB', 
    'Height_when_missing', 
    'Missing_date', 
    'Missing_location', 
    'Unique_traits', 
    'Other_info', 
    'Registration_datetime', 
    'geo_city',
    'geo_lat', 
    'geo_lng'
]]

In [ ]:
geocoded.to_csv("geocoded9000_to_10000.csv")

In [6]:
# after we geocoded all data, combine all 52 .csvs
df1 = pd.read_csv("geocoded0to1000.csv")
df2 = pd.read_csv("geocoded1000_to_2000.csv")
df3 = pd.read_csv("geocoded2000_to_3000.csv")
df4 = pd.read_csv("geocoded3000_to_4000.csv")
df5 = pd.read_csv("geocoded4000_to_5000.csv")
df6 = pd.read_csv("geocoded5000_to_6000.csv")
df7 = pd.read_csv("geocoded6000_to_7000.csv")
df8 = pd.read_csv("geocoded7000_to_8000.csv")
df9 = pd.read_csv("geocoded8000_to_9000.csv")
df10 = pd.read_csv("geocoded9000_to_10000.csv")
df11 = pd.read_csv("geocoded10K_to_11K.csv")
df12 = pd.read_csv("geocoded11K_to_12K.csv")
df13 = pd.read_csv("geocoded12K_to_13K.csv")
df14 = pd.read_csv("geocoded13K_to_14K.csv")
df15 = pd.read_csv("geocoded14K_to_15K.csv")
df16 = pd.read_csv("geocoded15K_to_16K.csv")
df17 = pd.read_csv("16K_to_17K.csv")
df18 = pd.read_csv("geocoded17K_to_18K.csv")
df19 = pd.read_csv("18K_to_19K.csv")
df20 = pd.read_csv("geocoded19K_to_20K.csv")
df21 = pd.read_csv("geocoded20K_to_21K.csv")
df22 = pd.read_csv("geocoded21K_22K.csv")
df23 = pd.read_csv("geocoded22K_to_23K.csv")
df24 = pd.read_csv("geocoded23_24K.csv")
df25 = pd.read_csv("24K_25K.csv")
df26 = pd.read_csv("25K_26K.csv")
df27 = pd.read_csv("26_27K.csv")
df28 = pd.read_csv("geocoded27K_28K.csv")
df29 = pd.read_csv("28_29K.csv")
df30 = pd.read_csv("29_30K.csv")
df31 = pd.read_csv("30_31K.csv")
df32 = pd.read_csv("31_32K.csv")
df33 = pd.read_csv("32_33K.csv")
df34 = pd.read_csv("33_34K.csv")
df35 = pd.read_csv("34_35K.csv")
df36 = pd.read_csv("35_36K.csv")
df37 = pd.read_csv("36_37K.csv")
df38 = pd.read_csv("37_38K.csv")
df39 = pd.read_csv("38_39K.csv")
df40 = pd.read_csv("39_40K.csv")
df41 = pd.read_csv("40_41K.csv")
df42 = pd.read_csv("41_42K.csv")
df43 = pd.read_csv("42_43K.csv")
df44 = pd.read_csv("43_44K.csv")
df45 = pd.read_csv("44_45K.csv")
df46 = pd.read_csv("45_46K.csv")
df47 = pd.read_csv("46_47K.csv")
df48 = pd.read_csv("47_48K.csv")
df49 = pd.read_csv("48_49K.csv")
df50 = pd.read_csv("49_50K.csv")
df51 = pd.read_csv("50_51K.csv")
df52 = pd.read_csv("51_end.csv")

In [9]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10,
         df11, df12, df13, df14, df15, df16, df17, df18, df19,
         df20, df21, df22, df23, df24, df25, df26, df27, df28,
         df29, df30, df31, df32, df33, df34, df35, df36, df37,
         df38, df39, df40, df41, df42, df43, df44, df45, df46,
         df47, df48, df49, df50, df51,df52]
df = pd.concat(frames)

In [252]:
# importing cleaned df with US city names changed to chinese cities
df_geocoded = pd.read_csv("US_name_remedied.csv")
df_geocoded = df_geocoded[[
    'ID', 
    'geo_city',
    'geo_lat', 
    'geo_lng'
]]

/Users/isabellachua/.pyenv/versions/3.8.2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (9,13,14,15,16,17,18,19,20,21,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [253]:
# replace chinese city names to english to join with china's shpfile
# for provinces, i didnt convert them to city level, so our maps will have some blanks
df_geocoded['geo_city'] = df_geocoded['geo_city'].replace({
    '陇南市': 'Longnan',
    '铜鼓': 'Tonggu',
    '铜仁市': 'Tongren',
    '重庆市': 'Chongqing',
    '遵义市': 'Zunyi',
    '达州市': 'Dazhou',
    '贵阳市': 'Guiyang',
    '西安市': "Xi'an",
    '苏州市': 'Suzhou',
    '芜湖市': 'Wuhu',
    '绍兴市': 'Shaoxing',
    '红河哈尼族彝族自治州': 'Honghe Hani and Yi',
    '石家庄市': 'Shijiazhuang',
    '眉县': 'Mei County',
    '澄江': 'Chengjiang',
    '潮州市': 'Chaozhou',
    '深圳市': 'Shenzhen',
    '海东地区': 'Haidong',
    '沈阳市': 'Shenyang',
    '桂林市': "Guilin",
    '松原市': 'Songyuan',
    '无锡市': 'Wuxi',
    '揭阳市': 'Jieyang',
    '户县': 'Huyi District',
    '广州市': 'Guangzhou',
    '崇左市': 'Chongzuo',
    '宁波市': 'Ningbo',
    '大连市': 'Dalian',
    '唐山市': 'Tangshan',
    '哈尔滨市': 'Harbin',
    '合肥市': 'Hefei',
    '厦门市': 'Xiamen',
    '南京市': "Nanjing",
    '佛山市': 'Foshan',
    '九江市': 'Jiujiang',
    '临沂市': 'Jieyang',
    '临沂市': 'Linyi',
    'zuishan': 'Shizuishan',
    'Yili Hasakezizhizhou': 'Ili Kazakh',
    'Yanbian Chaoxianzuzizhizhou': 'Yanbian Korean',
    'Yanbian Korean Autonomous Prefecture': "Yanbian Korean",
    'yan': 'Hubei',
    'Yaan': "Ya'an",
    'Xishuangbanna Daizuzizhizhou': 'Xishuangbanna Dai',
    'Xishuangbanna Dai Autonomous Prefecture': 'Xishuangbanna Dai',
    'Xilinguole Meng': 'Xilingol League',
    'Xiangxi Tujiazumiaozuzizhizhou': 'Xiangxi Tujia and Miao',
    'Xiangxi Tujia and Miao Autonomous Prefecture': 'Xiangxi Tujia and Miao', 
    'Wenshan Zhuangzumiaozuzizhizhou': 'Wenshan Zhuang and Miao',
    'Wenshan Zhuang and Miao Autonomous Prefecture': 'Wenshan Zhuang and Miao',
    'Xilinguole Meng': 'Xilingol League',
    'Tacheng Prefecture': 'Tacheng',
    'Tacheng Diqu': 'Tacheng',
    'Taian': "Tai'an",
    'Qianxinan Buyizumiaozuzizhizhou': 'Qianxinan Buyei and Miao',
    'Qianxinan Buyei and Miao Autonomous Prefecture': 'Qianxinan Buyei and Miao',
    'Qiannan Buyizumiaozuzizhizhou': 'Qiannan Buyei and Miao',
    'Qiannan Buyei and Miao Autonomous Prefecture': 'Qiannan Buyei and Miao',
    'Qiandongnan Miaozudongzuzizhizhou': 'Qiandongnan Miao and Dong',
    'Qiandongnan Miao and Dong Autonomous Prefecture': 'Qiandongnan Miao and Dong',
    'Puer': "Pu'er",
    "Pu'er City": "Pu'er",
    'Ordos City': 'Ordos',
    'Nujiang Lisu Autonomous Prefecture': 'Nujiang Lisu',
    'Ngawa Tibetan and Qiang Autonomous Prefecture': 'Ngawa Tibetan and Qiang Autonomous Prefecture',
    'nanjing city': 'Nanjing',
    'Maanshan':"Ma'anshan",
    'Lvliang Shi': 'Luliang',
    'Lvliang': 'Luliang',
    'Lüliang': 'Luliang',
    'Luan': "Lu'an",
    'Liangshan Yizuzizhizhou': 'Liangshan Yi',
    'Liangshan Yi Autonomous Prefecture': 'Liangshan Yi',
    'Kizilsu Kyrgyz Autonomous Prefecture': 'Kizilsu Kirghiz',
    'Kelamayi': 'Karamay',
    'Kashi Diqu': 'Kashgar Prefecture',
    'Jilin City': 'Jilin',
    'jiazhuang': 'Shijiazhuang',
    'Jiayuguan City': 'Jiayuguan',
    'Jian': "Ji'an",
    'Ili': 'Ili Kazakh',
    'Hulunbeier': 'Hulunbuir',
    'Huhehaote': 'Hohhot',
    'Huangshan City': 'Huangshan',
    'Huangnan Tibetan Autonomous Prefecture': 'Huangnan Tibetan',
    'Huaian': "Huai'an",
    'Hotan Prefecture': 'Hotan',
    'Honghe Hanizuyizuzizhizhou': 'Honghe Hani and Yi',
    'Honghe Hani and Yi Autonomous Prefecture': 'Honghe Hani and Yi',
    'Hetian Diqu': "Hotan Prefecture",
    'Haixi Mongol and Tibetan Autonomous Prefecture': 'Haixi Mongol and Tibetan',
    'Haixi Mengguzuzangzuzizhizhou': 'Haixi Mongol and Tibetan',
    'Hainan Tibetan Autonomous Prefecture': 'Hainan Tibetan',
    'Hainan Zangzuzizhizhou': 'Hainan Tibetan',
    'Haibei Zangzuzizhizhou': 'Haibei Tibetan',
    'Haibei': 'Haibei Tibetan',
    'Haerbin': "Harbin",
    'Guoluo Zangzuzizhizhou': 'Golog Tibetan',
    'Guangan': "Guang'an",
    'gatse': 'Shigatse',
    'Garzê Tibetan Autonomous Prefecture': 'Garzê Tibetan',
    "Ganzi Zangzuzizhizhou": 'Garzê Tibetan',
    'Gannan Tibetan Autonomous Prefecture': 'Gannan Tibetan',
    'Enshi City': 'Enshi Tujia and Miao',
    'Enshi Tujiazumiaozuzizhizhou': 'Enshi Tujia and Miao',
    'Eerduosi': 'Ordos',
    'Dehong': "Dehong Dai and Jingpo",
    'Dehong Dai and Jingpo Autonomous Prefecture': "Dehong Dai and Jingpo",
    'Daxinganling Diqu': 'Heilongjiang',
    "Daxing'anling Prefecture": 'Heilongjiang',
    'Dali Baizuzizhizhou': 'Dali Bai',
    'Dali': 'Dali Bai',
    'Chuxiong Yizuzizhizhou': 'Chuxiong Yi',
    'Chuxiong Yi Autonomous Prefecture': 'Chuxiong Yi',
    "Chaoyang, Liaoning": 'Chaoyang',
    'Changji Huizuzizhizhou': 'Changji Hui',
    'Changji Hui Autonomous Prefecture': 'Changji Hui',
    'Boertala Mengguzizhizhou': 'Bortala Mongol Autonomous Prefecture',
    'Bayinguoleng Mengguzizhizhou': 'Bayingolin Mongol Autonomous Prefecture',
    'Bayannaoer': 'Bayannur',
    'Alxa League': 'Alxa',
    'Altay Prefecture': 'Altay',
    "Aletai Diqu": 'Altay',
    "Alashan Meng": 'Alxa',
    'Aksu Prefecture': 'Aksu',
    'Akesu Diqu': 'Aksu',
    'Aba Zangzuqiangzuzizhizhou': 'Ngawa Tibetan and Qiang',
    'Xian': "Xi'an",
    'Qiqihaer': 'Qiqihar'
    
})

In [255]:
df_geocoded.head()

,ID,geo_city,geo_lat,geo_lng
0,260659,Ngawa Tibetan and Qiang,32.905769,101.7000695
1,200123,Ngawa Tibetan and Qiang,31.679062,103.852127
2,6121,Ngawa Tibetan and Qiang,31.4774575,103.5875882
3,549931,Aksu,41.169599,80.255364
4,531621,Aksu,41.7977072,81.8780182


In [258]:
# merge csv with other info with geocoded df_geocoded
df1 = pd.read_csv("cleaned_8Aug.csv")
df1 = df1.drop(columns=['Unnamed: 0'])
df1.head()

,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime
0,557216,蒋玉梅（送养）,女,1988年12月01日,50厘米左右,1988年12月16日,"湖南省,郴州市,桂阳县李家湾13号（电影院后面）",无,无,2021/8/5 23:54:29
1,557160,韩宇辰 尔萨,男,2018年09月11日,105厘米左右,2021年08月02日,"青海省,海东地区,循化县积石镇伊麻目村141号",小孩左小腿有开水烫伤疤痕，微胖，会叫爸妈爷爷奶奶,NaN,2021/8/5 15:48:19
2,556774,宦方艳,女,2001年09月01日,60厘米左右,2003年10月18日,"贵州省,遵义市,仁怀市大坝镇",无,无,2021/8/3 14:06:30
3,556769,孙老三,女,2001年11月27日,45厘米左右,2001年11月28日,"贵州省,遵义市,仁怀市仓头社区",无,无,2021/8/3 13:46:35
4,556740,王祥申,男,1993年04月05日,未知,1994年02月12日,"云南省,昭通市,威信县庙沟镇扎实沟村",肚脐附近有颗黑痣,NaN,2021/8/3 9:32:44


In [259]:
df = df1.merge(df_geocoded, left_on="ID", right_on="ID")

In [261]:
df.to_csv("geocoded_without_datetime.csv")

In [262]:
df

,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng
0,557216,蒋玉梅（送养）,女,1988年12月01日,50厘米左右,1988年12月16日,"湖南省,郴州市,桂阳县李家湾13号（电影院后面）",无,无,2021/8/5 23:54:29,Chenzhou,25.754116,112.734141
1,557160,韩宇辰 尔萨,男,2018年09月11日,105厘米左右,2021年08月02日,"青海省,海东地区,循化县积石镇伊麻目村141号",小孩左小腿有开水烫伤疤痕，微胖，会叫爸妈爷爷奶奶,NaN,2021/8/5 15:48:19,Haidong,35.869702,102.424422
2,556774,宦方艳,女,2001年09月01日,60厘米左右,2003年10月18日,"贵州省,遵义市,仁怀市大坝镇",无,无,2021/8/3 14:06:30,Zunyi,28.019659,106.417
3,556769,孙老三,女,2001年11月27日,45厘米左右,2001年11月28日,"贵州省,遵义市,仁怀市仓头社区",无,无,2021/8/3 13:46:35,Zunyi,27.79165,106.400342
4,556740,王祥申,男,1993年04月05日,未知,1994年02月12日,"云南省,昭通市,威信县庙沟镇扎实沟村",肚脐附近有颗黑痣,NaN,2021/8/3 9:32:44,Zhaotong,27.758203,104.821717
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,2193,佟月,男,1997年01月01日,未知,2000年01月01日,北京,和小朋友在外面玩被陌生人带走,（流浪天涯跟进）,\n,Beijing,39.9041999,116.4073963
52003,2192,曾琪雅,男,1992年10月14日,未知,1999年03月04日,湖南省邵东县佘田桥镇街上,曾琪雅（小名万万），湖南省邵东县佘田桥镇人，于1999年3月4日在湖南省邵东县佘田桥...,（漆彩生活跟进）,\n,Shaoyang,27.120864,111.949798
52004,2191,谷志伟,男,1986年08月29日,未知,1989年10月29日,河南省长葛市烟草公司门口,小时候做过疝气手术，留有刀疤,(小梅跟进）,\n,Xuchang Shi,34.2309312,113.935709
52005,2188,李鑫*,男,1999年01月01日,未知,2004年02月02日,昆明小坝附近,李兴，男，5岁，身高120厘米左右，头较大，圆脸，上牙稀疏较黑，身穿蓝色（胸前有两白...,NaN,\n,Kunming,24.944467,103.293835


In [266]:
# change ID from datetime to str so that we can drop duplicates accurately
df["ID"] = df["ID"].astype(str)

In [267]:
df.shape

(52007, 13)

In [268]:
df = df.drop_duplicates()

In [269]:
# checking if all rows are unique
print(df.duplicated().value_counts())
df

False    52007
dtype: int64


,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng
0,557216,蒋玉梅（送养）,女,1988年12月01日,50厘米左右,1988年12月16日,"湖南省,郴州市,桂阳县李家湾13号（电影院后面）",无,无,2021/8/5 23:54:29,Chenzhou,25.754116,112.734141
1,557160,韩宇辰 尔萨,男,2018年09月11日,105厘米左右,2021年08月02日,"青海省,海东地区,循化县积石镇伊麻目村141号",小孩左小腿有开水烫伤疤痕，微胖，会叫爸妈爷爷奶奶,NaN,2021/8/5 15:48:19,Haidong,35.869702,102.424422
2,556774,宦方艳,女,2001年09月01日,60厘米左右,2003年10月18日,"贵州省,遵义市,仁怀市大坝镇",无,无,2021/8/3 14:06:30,Zunyi,28.019659,106.417
3,556769,孙老三,女,2001年11月27日,45厘米左右,2001年11月28日,"贵州省,遵义市,仁怀市仓头社区",无,无,2021/8/3 13:46:35,Zunyi,27.79165,106.400342
4,556740,王祥申,男,1993年04月05日,未知,1994年02月12日,"云南省,昭通市,威信县庙沟镇扎实沟村",肚脐附近有颗黑痣,NaN,2021/8/3 9:32:44,Zhaotong,27.758203,104.821717
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,2193,佟月,男,1997年01月01日,未知,2000年01月01日,北京,和小朋友在外面玩被陌生人带走,（流浪天涯跟进）,\n,Beijing,39.9041999,116.4073963
52003,2192,曾琪雅,男,1992年10月14日,未知,1999年03月04日,湖南省邵东县佘田桥镇街上,曾琪雅（小名万万），湖南省邵东县佘田桥镇人，于1999年3月4日在湖南省邵东县佘田桥...,（漆彩生活跟进）,\n,Shaoyang,27.120864,111.949798
52004,2191,谷志伟,男,1986年08月29日,未知,1989年10月29日,河南省长葛市烟草公司门口,小时候做过疝气手术，留有刀疤,(小梅跟进）,\n,Xuchang Shi,34.2309312,113.935709
52005,2188,李鑫*,男,1999年01月01日,未知,2004年02月02日,昆明小坝附近,李兴，男，5岁，身高120厘米左右，头较大，圆脸，上牙稀疏较黑，身穿蓝色（胸前有两白...,NaN,\n,Kunming,24.944467,103.293835


In [270]:
# make sure there're no extra spacings
df['DOB'] = df['DOB'].str.strip()
df['Missing_date'] = df['Missing_date'].str.strip()
df['geo_city'] = df['geo_city'].str.strip()
df['geo_lat'] = df['geo_lat'].str.strip()
df['geo_lng'] = df['geo_lng'].str.strip()

In [272]:
# convert "未知时间" (they dont know the date) to NaN values for columns 'DOB' and 'Missing_date'
# I know to do this bc an error occurred when I tried to make the two columns datetime

import numpy as np
df['DOB'] = df['DOB'].replace({
    "未知时间" : np.nan
})

In [273]:
df['Missing_date'] = df['Missing_date'].replace({
    "未知时间" : np.nan,
    "未知" : np.nan
})

In [274]:
# checking if the replacement works
df.sort_values(by="Missing_date", ascending=True)

,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng
13080,372079,未取名（遗弃）,女,1900年01月14日,未知,1900年01月14日,"江苏省,泰州市, 海陵区",刚出生42天身上没有胎记,姐，你是我二姐，当时把你送掉母亲也是万般不舍，不然也不会养到42天才送人，家里也一直瞒着我，...,2019/1/30 19:02:34,Taizhou,32.491016,119.919424
10976,394987,不详（送养）日期需核实后修改,女,1900年06月02日,未知,1900年06月02日,"河南省,南阳市, 新野县沙堰镇",当时一个月,当时一个月，没有记忆,2019/6/2 13:16:05,Nanyang,32.604386,112.461137
11584,388263,卢（照片是姐姐）（送养）,女,1900年05月26日,未知,1900年06月26日,"黑龙江省,齐齐哈尔市,梅里斯区达呼店镇瑞廷村八队",女，二十多天被抱走,NaN,2019/4/26 10:12:17,Qiqihar,47.531757,123.821805
10565,399928,王孩,男,1900年02月26日,60厘米左右,1901年04月26日,"河南省,南阳市,南召县城郊乡马鹿岗村",无,无,2019/6/26 21:36:15,Nanyang,33.474093,112.437563
9567,411072,赵海珊（送养）,女,1902年01月01日,未知,1902年01月01日,"河南省,漯河市,源汇区滨河路",女孩，我妹妹，当年计划生育生下来送人了，现在父母年迈，父亲重病，想见见小女儿，弥补一下她。,女孩，我妹妹，当年计划生育生下来送人了，现在父母年迈，父亲重病，想见见小女儿，弥补一下她。,2019/7/31 20:25:58,Luohe Shi,33.570512,114.0223329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50621,12950,朱伟杰,男,1989年01月01日,110厘米左右,NaN,河南省周口市鹿邑县玄武镇,右脚脚趾曾断伤（打面机打断右脚四个脚趾）,（独饮跟进）,2009/12/18 15:48:49,Zhoukou,33.968059,115.27651
50717,12242,朱华龙,男,1980年12月11日,130厘米左右,NaN,湖北省武汉市航空路,\n,（二月格桑 跟进）,2009/12/4 23:52:07,Wuhan Shi,30.5848418,114.261422
51871,3314,失踪女孩,女,NaN,未知,NaN,\n,"荆楚网消息 (楚天都市报) 编者的话,一场冬雨，气温骤降。街头的圣诞树和火锅店里的腾...",（都市浪人跟进）,2007/12/11 10:12:29,NaN,NaN,NaN
51968,2391,易晨,男,1998年01月01日,150厘米左右,NaN,陕西省西安市未央区西三村,\n,（联系不到寻亲人，请寻亲人看到信息后尽快与网站联系，网站电话：0435-3338090）（帝...,\n,Xi'an,34.372851,108.930924


In [275]:
import datetime as dt
from datetime import datetime

In [276]:
# create a function that converts the date columns into datetime using %Y%m%d
# and convert remaining that dont conform into NaN values
def chin_to_eng(datename):
    try:
        cleaned_date = datename.strip().replace("年","").replace("月","").replace("日","")
        new_date = dt.datetime.strptime(cleaned_date,"%Y%m%d") #2021-08-13
        return new_date
    except AttributeError:
        return np.nan

In [277]:
df['DOB'] = df['DOB'].apply(chin_to_eng)

In [278]:
df['Missing_date'] = df['Missing_date'].apply(chin_to_eng)

In [280]:
# remove "Shi" from geo_city column; they basically mean city, which is redundant here
df['geo_city'] = df['geo_city'].str.replace("Shi", '')

In [281]:
# find how old were they when they went missing
df['Missing_age'] = df['Missing_date'] - df['DOB']

In [283]:
# remove "days" from result
# because we want it to be int to bin them
df['Missing_age'] = df['Missing_age'].astype('timedelta64[D]')

In [284]:
# extracting year, month, and day where they go missing
df['Year_missing'] = pd.DatetimeIndex(df['Missing_date']).year
df['Month_missing'] = pd.DatetimeIndex(df['Missing_date']).month
df['Day_missing'] = pd.DatetimeIndex(df['Missing_date']).day

In [297]:
# how many days missing since today (2021-8-9) // 9 Aug 2021
today = datetime.today().strftime("%Y-%m-%d")

In [298]:
today = pd.to_datetime('20210809', format='%Y-%m-%d', errors='ignore')
today

Timestamp('2021-08-09 00:00:00')

In [299]:
df['Days_missing'] = today - df['Missing_date'] 

In [301]:
df['Days_missing'] = df['Days_missing'].astype('timedelta64[D]')

In [302]:
df

,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng,Missing_age,Year_missing,Month_missing,Day_missing,Days_missing
0,557216,蒋玉梅（送养）,女,1988-12-01,50厘米左右,1988-12-16,"湖南省,郴州市,桂阳县李家湾13号（电影院后面）",无,无,2021/8/5 23:54:29,Chenzhou,25.754116,112.734141,15.0,1988.0,12.0,16.0,11924.0
1,557160,韩宇辰 尔萨,男,2018-09-11,105厘米左右,2021-08-02,"青海省,海东地区,循化县积石镇伊麻目村141号",小孩左小腿有开水烫伤疤痕，微胖，会叫爸妈爷爷奶奶,NaN,2021/8/5 15:48:19,Haidong,35.869702,102.424422,1056.0,2021.0,8.0,2.0,7.0
2,556774,宦方艳,女,2001-09-01,60厘米左右,2003-10-18,"贵州省,遵义市,仁怀市大坝镇",无,无,2021/8/3 14:06:30,Zunyi,28.019659,106.417,777.0,2003.0,10.0,18.0,6505.0
3,556769,孙老三,女,2001-11-27,45厘米左右,2001-11-28,"贵州省,遵义市,仁怀市仓头社区",无,无,2021/8/3 13:46:35,Zunyi,27.79165,106.400342,1.0,2001.0,11.0,28.0,7194.0
4,556740,王祥申,男,1993-04-05,未知,1994-02-12,"云南省,昭通市,威信县庙沟镇扎实沟村",肚脐附近有颗黑痣,NaN,2021/8/3 9:32:44,Zhaotong,27.758203,104.821717,313.0,1994.0,2.0,12.0,10040.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,2193,佟月,男,1997-01-01,未知,2000-01-01,北京,和小朋友在外面玩被陌生人带走,（流浪天涯跟进）,\n,Beijing,39.9041999,116.4073963,1095.0,2000.0,1.0,1.0,7891.0
52003,2192,曾琪雅,男,1992-10-14,未知,1999-03-04,湖南省邵东县佘田桥镇街上,曾琪雅（小名万万），湖南省邵东县佘田桥镇人，于1999年3月4日在湖南省邵东县佘田桥...,（漆彩生活跟进）,\n,Shaoyang,27.120864,111.949798,2332.0,1999.0,3.0,4.0,8194.0
52004,2191,谷志伟,男,1986-08-29,未知,1989-10-29,河南省长葛市烟草公司门口,小时候做过疝气手术，留有刀疤,(小梅跟进）,\n,Xuchang,34.2309312,113.935709,1157.0,1989.0,10.0,29.0,11607.0
52005,2188,李鑫*,男,1999-01-01,未知,2004-02-02,昆明小坝附近,李兴，男，5岁，身高120厘米左右，头较大，圆脸，上牙稀疏较黑，身穿蓝色（胸前有两白...,NaN,\n,Kunming,24.944467,103.293835,1858.0,2004.0,2.0,2.0,6398.0


In [303]:
df.to_csv("ready_for_viz.csv")

In [295]:
df['geo_city'].value_counts()

Guiyang                704
Guiyang                621
Wuhan                  556
Bijie                  539
Zunyi                  497
                      ... 
2011/8/6 10:17:20        1
2011/6/6 10:26:54        1
Huyi District            1
2014/12/10 21:06:49      1
2009/9/21 4:58:45        1
Name: geo_city, Length: 717, dtype: int64

### stuff i did to get US_name_remedied

In [184]:
df.loc[df['geo_city'] == "Qiqihaer"]

,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng,Missing_age,Year_missing,Month_missing,Day_missing
67,544232,不详（送养）,女,1979-12-09,未知,1979-12-09,"黑龙江省,齐齐哈尔市,黑龙江呼盟人民医院",不详,NaN,2021/6/4 16:38:05,Qiqihaer,48.484028,124.887274,0.0,1979.0,12.0,9.0
386,537941,不知道（送养）,女,1973-07-22,未知,1973-07-22,"黑龙江省,齐齐哈尔市,依安县人民医院",生产时大出血，被大夫送人，女孩，抱家里一个，六个月生病没有了,妈妈姓周，爸爸姓李,2021/5/8 22:27:28,Qiqihaer,47.892285,125.304754,0.0,1973.0,7.0,22.0
146,523994,未取名（送养）,女,1969-03-26,30厘米左右,1969-03-26,"黑龙江省,齐齐哈尔市,建华区203医院妇产科",刚出生女婴,无,2021/3/3 16:30:00,Qiqihaer,47.345510,123.958309,0.0,1969.0,3.0,26.0
572,519866,没有（送养）,男,1981-05-16,60厘米左右,1981-05-16,"黑龙江省,齐齐哈尔市,讷河市卫东街19组",体重9斤，耳垂厚,NaN,2021/1/11 17:49:07,Qiqihaer,48.486848,124.880549,0.0,1981.0,5.0,16.0
120,506177,女孩（送养）,女,1989-08-19,50厘米左右,1989-09-27,"黑龙江省,齐齐哈尔市,火车站附近",不详,NaN,2020/10/27 11:15:55,Qiqihaer,48.651001,125.006427,39.0,1989.0,9.0,27.0
157,480030,张君（送养）,女,1985-10-12,60厘米左右,1986-01-17,"黑龙江省,齐齐哈尔市,黑龙江省齐齐哈尔市克东县新农公社新泉大队",头旋及断掌纹不详，单眼皮。,NaN,2020/6/1 16:18:00,Qiqihaer,48.046433,126.265624,97.0,1986.0,1.0,17.0
462,475745,姚银花（送养）,女,1990-12-17,50厘米左右,1991-01-17,"黑龙江省,齐齐哈尔市,富拉尔基区铁西",不详,NaN,2020/5/3 19:57:04,Qiqihaer,47.211909,123.624039,31.0,1991.0,1.0,17.0
582,450614,朋朋,男,1994-05-05,未知,1995-12-19,黑龙江省齐齐哈尔市火车站候车室,左胳膊或者右胳膊上有个痣。,NaN,2020/1/19 11:22:26,Qiqihaer,47.339265,123.995723,593.0,1995.0,12.0,19.0
596,450435,不详（送养）,女,1977-01-17,50厘米左右,1977-01-17,"黑龙江省,齐齐哈尔市,建华区, 商职医院",头发黑、皮肤白、眼睛大、圆脸,NaN,2020/1/17 9:34:24,Qiqihaer,47.364842,123.966374,0.0,1977.0,1.0,17.0
754,446855,仪小兰（送养）,女,1983-07-19,未知,1983-07-20,"黑龙江省,大庆市,拜泉县人民医院",婴儿早上下的小雨身包灰底白格女士半截大衣内科病房抱出来的出生三天后养父送给老李头三百块钱,NaN,2020/1/2 9:56:47,Qiqihaer,47.603947,126.096017,1.0,1983.0,7.0,20.0


In [ ]:
# date_list = []
# for only_dates in df['Registration_datetime']:
#     try:
#         date = pd.to_datetime(only_dates).date()
#         date_list.append(date)
#     except:
#         pass

In [93]:
# clean the "geo_city" column
df["geo_city"] = df["geo_city"].str.strip()
# drop "Shi" from all; it just means city
df['geo_city'] = df['geo_city'].str.replace("Shi", '')

In [94]:
cities_untranslated = pd.DataFrame(cities)
cities_untranslated.head(5)

,geo_city
Guiyang,1326
Chengdu,928
Wuhan,856
Zunyi,788
Guangzhou,722


In [95]:
# save to csv to identify all cities spelt in chinese
cities_untranslated.to_csv("untranslated_cities.csv")

### Prepping data for choropleth map

In [307]:
df = pd.read_csv("weirdcities_4am.csv")

In [308]:
df['geo_city'] = df['geo_city'].replace({
    '陇南市': 'Longnan',
    '铜鼓': 'Tonggu',
    '铜仁市': 'Tongren',
    '重庆市': 'Chongqing',
    '遵义市': 'Zunyi',
    '达州市': 'Dazhou',
    '贵阳市': 'Guiyang',
    '西安市': "Xi'an",
    '苏州市': 'Suzhou',
    '芜湖市': 'Wuhu',
    '绍兴市': 'Shaoxing',
    '红河哈尼族彝族自治州': 'Honghe Hani and Yi',
    '石家庄市': 'Shijiazhuang',
    '眉县': 'Mei County',
    '澄江': 'Chengjiang',
    '潮州市': 'Chaozhou',
    '深圳市': 'Shenzhen',
    '海东地区': 'Haidong',
    '沈阳市': 'Shenyang',
    '桂林市': "Guilin",
    '松原市': 'Songyuan',
    '无锡市': 'Wuxi',
    '揭阳市': 'Jieyang',
    '户县': 'Huyi District',
    '广州市': 'Guangzhou',
    '崇左市': 'Chongzuo',
    '宁波市': 'Ningbo',
    '大连市': 'Dalian',
    '唐山市': 'Tangshan',
    '哈尔滨市': 'Harbin',
    '合肥市': 'Hefei',
    '厦门市': 'Xiamen',
    '南京市': "Nanjing",
    '佛山市': 'Foshan',
    '九江市': 'Jiujiang',
    '临沂市': 'Jieyang',
    '临沂市': 'Linyi',
    'zuishan': 'Shizuishan',
    'Yili Hasakezizhizhou': 'Ili Kazakh',
    'Yanbian Chaoxianzuzizhizhou': 'Yanbian Korean',
    'Yanbian Korean Autonomous Prefecture': "Yanbian Korean",
    'yan': 'Hubei',
    'Yaan': "Ya'an",
    'Xishuangbanna Daizuzizhizhou': 'Xishuangbanna Dai',
    'Xishuangbanna Dai Autonomous Prefecture': 'Xishuangbanna Dai',
    'Xilinguole Meng': 'Xilingol League',
    'Xiangxi Tujiazumiaozuzizhizhou': 'Xiangxi Tujia and Miao',
    'Xiangxi Tujia and Miao Autonomous Prefecture': 'Xiangxi Tujia and Miao', 
    'Wenshan Zhuangzumiaozuzizhizhou': 'Wenshan Zhuang and Miao',
    'Wenshan Zhuang and Miao Autonomous Prefecture': 'Wenshan Zhuang and Miao',
    'Xilinguole Meng': 'Xilingol League',
    'Tacheng Prefecture': 'Tacheng',
    'Tacheng Diqu': 'Tacheng',
    'Taian': "Tai'an",
    'Qianxinan Buyizumiaozuzizhizhou': 'Qianxinan Buyei and Miao',
    'Qianxinan Buyei and Miao Autonomous Prefecture': 'Qianxinan Buyei and Miao',
    'Qiannan Buyizumiaozuzizhizhou': 'Qiannan Buyei and Miao',
    'Qiannan Buyei and Miao Autonomous Prefecture': 'Qiannan Buyei and Miao',
    'Qiandongnan Miaozudongzuzizhizhou': 'Qiandongnan Miao and Dong',
    'Qiandongnan Miao and Dong Autonomous Prefecture': 'Qiandongnan Miao and Dong',
    'Puer': "Pu'er",
    "Pu'er City": "Pu'er",
    'Ordos City': 'Ordos',
    'Nujiang Lisu Autonomous Prefecture': 'Nujiang Lisu',
    'Ngawa Tibetan and Qiang Autonomous Prefecture': 'Ngawa Tibetan and Qiang Autonomous Prefecture',
    'nanjing city': 'Nanjing',
    'Maanshan':"Ma'anshan",
    'Lvliang Shi': 'Luliang',
    'Lvliang': 'Luliang',
    'Lüliang': 'Luliang',
    'Luan': "Lu'an",
    'Liangshan Yizuzizhizhou': 'Liangshan Yi',
    'Liangshan Yi Autonomous Prefecture': 'Liangshan Yi',
    'Kizilsu Kyrgyz Autonomous Prefecture': 'Kizilsu Kirghiz',
    'Kelamayi': 'Karamay',
    'Kashi Diqu': 'Kashgar Prefecture',
    'Jilin City': 'Jilin',
    'jiazhuang': 'Shijiazhuang',
    'Jiayuguan City': 'Jiayuguan',
    'Jian': "Ji'an",
    'Ili': 'Ili Kazakh',
    'Hulunbeier': 'Hulunbuir',
    'Huhehaote': 'Hohhot',
    'Huangshan City': 'Huangshan',
    'Huangnan Tibetan Autonomous Prefecture': 'Huangnan Tibetan',
    'Huaian': "Huai'an",
    'Hotan Prefecture': 'Hotan',
    'Honghe Hanizuyizuzizhizhou': 'Honghe Hani and Yi',
    'Honghe Hani and Yi Autonomous Prefecture': 'Honghe Hani and Yi',
    'Hetian Diqu': "Hotan Prefecture",
    'Haixi Mongol and Tibetan Autonomous Prefecture': 'Haixi Mongol and Tibetan',
    'Haixi Mengguzuzangzuzizhizhou': 'Haixi Mongol and Tibetan',
    'Hainan Tibetan Autonomous Prefecture': 'Hainan Tibetan',
    'Hainan Zangzuzizhizhou': 'Hainan Tibetan',
    'Haibei Zangzuzizhizhou': 'Haibei Tibetan',
    'Haibei': 'Haibei Tibetan',
    'Haerbin': "Harbin",
    'Guoluo Zangzuzizhizhou': 'Golog Tibetan',
    'Guangan': "Guang'an",
    'gatse': 'Shigatse',
    'Garzê Tibetan Autonomous Prefecture': 'Garzê Tibetan',
    "Ganzi Zangzuzizhizhou": 'Garzê Tibetan',
    'Gannan Tibetan Autonomous Prefecture': 'Gannan Tibetan',
    'Enshi City': 'Enshi Tujia and Miao',
    'Enshi Tujiazumiaozuzizhizhou': 'Enshi Tujia and Miao',
    'Eerduosi': 'Ordos',
    'Dehong': "Dehong Dai and Jingpo",
    'Dehong Dai and Jingpo Autonomous Prefecture': "Dehong Dai and Jingpo",
    'Daxinganling Diqu': 'Heilongjiang',
    "Daxing'anling Prefecture": 'Heilongjiang',
    'Dali Baizuzizhizhou': 'Dali Bai',
    'Dali': 'Dali Bai',
    'Chuxiong Yizuzizhizhou': 'Chuxiong Yi',
    'Chuxiong Yi Autonomous Prefecture': 'Chuxiong Yi',
    "Chaoyang, Liaoning": 'Chaoyang',
    'Changji Huizuzizhizhou': 'Changji Hui',
    'Changji Hui Autonomous Prefecture': 'Changji Hui',
    'Boertala Mengguzizhizhou': 'Bortala Mongol Autonomous Prefecture',
    'Bayinguoleng Mengguzizhizhou': 'Bayingolin Mongol Autonomous Prefecture',
    'Bayannaoer': 'Bayannur',
    'Alxa League': 'Alxa',
    'Altay Prefecture': 'Altay',
    "Aletai Diqu": 'Altay',
    "Alashan Meng": 'Alxa',
    'Aksu Prefecture': 'Aksu',
    'Akesu Diqu': 'Aksu',
    'Aba Zangzuqiangzuzizhizhou': 'Ngawa Tibetan and Qiang',
    'Xian': "Xi'an",
    'Qiqihaer': 'Qiqihar'
    
})

In [7]:
# df.to_csv("combined_4am.csv")

In [4]:
df = pd.read_csv("combined_4am.csv")

In [316]:
cities = pd.DataFrame(cities)

In [318]:
cities.to_csv("cities_only.csv") # check against this and remove weird names

In [12]:
df.loc[df['geo_city'] == "Shanghai Shi"]

,Unnamed: 0,Unnamed: 0.1,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng,Missing_age(days),Year_missing,Month_missing,Day_missing,Days_missing


In [10]:
# need to replace city names again bc this includes new geocoded data
# that the prevd .replace didnt catch
df['geo_city'] = df['geo_city'].replace({
    'Chongqing Shi': 'Chongqing',
    'Shanghai Shi': 'Shanghai',
    'Beijing Shi': 'Beijing',
    'Ngawa Tibetan and Qiang Autonomous Prefecture': 'Ngawa Tibetan and Qiang',
    'Guangdong Province': 'Guangdong',
    'Guangdong Sheng': 'Guangdong',
    'Tianjin Shi': 'Tianjin',
    'Xinjiang Weiwuerzizhiqu': 'Xinjiang Uygur',
    'Tianjin Shi': 'Tianjin',
    'Dehong Daizujingpozuzizhizhou': 'Dehong Dai and Jingpo',
    'Guizhou Sheng': 'Guizhou',
    'Shanghái': 'Shanghai',
    'Guangxi Zhuangzuzizhiqu': 'Guangxi',
    'Dongguanzhuangcun': 'Dongguan',
    'Yunnan Sheng': 'Yunnan',
    'Jiangsu Sheng': 'Jiangsu',
    'Sichuan Sheng': 'Sichuan',
    'Guangdong Sheng': 'Guangdong',
    'Bejing': 'Beijing'
})

In [15]:
cities = df['geo_city'].value_counts()

In [17]:
cities_clean = pd.DataFrame(cities)

In [18]:
# hopefully ready to join this to the shpfile!
cities_clean.to_csv("cleancities_10Aug.csv")

In [19]:
df

,Unnamed: 0,Unnamed: 0.1,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng,Missing_age(days),Year_missing,Month_missing,Day_missing,Days_missing
0,0,18964,296933,卢雅琪,女,2002-12-29,165厘米左右,2017-09-22,"重庆市,市辖区,渝中区解放碑",上下牙龈有铁丝绵被。,NaN,2017/11/11 21:32:11,Chongqing,29.5572164,106.5771192,5381.0,2017.0,9.0,22.0,1417.0
1,1,28287,199646,妹妹（遗弃）,女,1984-06-20,未知,1984-07-25,"重庆市,市辖区,解放碑附近",刚出生7天,NaN,2016/6/21 16:35:19,Chongqing,29.5572164,106.5771192,35.0,1984.0,7.0,25.0,13529.0
2,2,41706,78228,黄雨锋,男,1991-12-11,80厘米左右,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,（燕子跟进）,2014/2/19 19:30:58,Shenzhen,22.53190021,114.2051181,1428.0,1995.0,11.0,8.0,9406.0
3,3,31011,174869,林炜（送养）,男,1996-09-16,未知,1996-09-17,"北京市,市辖区,天坛",无法记清,NaN,2016/3/18 9:23:25,Beijing,39.8807799,116.4191967,1.0,1996.0,9.0,17.0,9092.0
4,4,34602,144453,赵淑亮,女,1987-04-12,110厘米左右,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",大眼，短发额头有些小疤痕,（朵朵跟进）她那时说话不清楚,2015/9/2 7:05:07,Beijing,39.8713,116.3998,1331.0,1990.0,12.0,3.0,11207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,52002,49489,24275,李鑫,女,1994-11-03,未知,2011-01-01,\n,头顶上有一个红色的肉痔，大概有胡豆那么大，在我们家时叫何鑫，是我们家代养的，也称呼我们的父母...,（家长送养）在我们家时叫何鑫，是我们家代养的，也称呼我们的父母为爸爸、妈妈，我爸爸是开车的，...,2011/4/3 21:22:02,NaN,NaN,NaN,5903.0,2011.0,1.0,1.0,3873.0
52003,52003,49581,23443,吴文达,男,1988-09-08,155厘米左右,1999-02-10,澳门,"身材高瘦,单眼皮,大耳朵",NaN,2011/3/5 0:02:58,Macau,22.198745,113.543873,3807.0,1999.0,2.0,10.0,8216.0
52004,52004,49611,23233,寻找妹妹,女,1988-10-01,未知,1988-01-01,湖南省襄樊市,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,家长送养,2011/2/27 20:19:20,Xiangyang,32.006871,112.121945,-274.0,1988.0,1.0,1.0,12274.0
52005,52005,49661,22932,王梦楠,女,1994-01-24,145厘米左右,2001-12-28,天津市武进区,\n,（鹤姐跟进）在早晨上学的时候失踪，失踪时上小学1年级。,2011/2/21 13:56:55,Tianjin,39.341163,117.361784,2895.0,2001.0,12.0,28.0,7164.0


In [20]:
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['Unnamed: 0.1'])

In [21]:
df

,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng,Missing_age(days),Year_missing,Month_missing,Day_missing,Days_missing
0,296933,卢雅琪,女,2002-12-29,165厘米左右,2017-09-22,"重庆市,市辖区,渝中区解放碑",上下牙龈有铁丝绵被。,NaN,2017/11/11 21:32:11,Chongqing,29.5572164,106.5771192,5381.0,2017.0,9.0,22.0,1417.0
1,199646,妹妹（遗弃）,女,1984-06-20,未知,1984-07-25,"重庆市,市辖区,解放碑附近",刚出生7天,NaN,2016/6/21 16:35:19,Chongqing,29.5572164,106.5771192,35.0,1984.0,7.0,25.0,13529.0
2,78228,黄雨锋,男,1991-12-11,80厘米左右,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,（燕子跟进）,2014/2/19 19:30:58,Shenzhen,22.53190021,114.2051181,1428.0,1995.0,11.0,8.0,9406.0
3,174869,林炜（送养）,男,1996-09-16,未知,1996-09-17,"北京市,市辖区,天坛",无法记清,NaN,2016/3/18 9:23:25,Beijing,39.8807799,116.4191967,1.0,1996.0,9.0,17.0,9092.0
4,144453,赵淑亮,女,1987-04-12,110厘米左右,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",大眼，短发额头有些小疤痕,（朵朵跟进）她那时说话不清楚,2015/9/2 7:05:07,Beijing,39.8713,116.3998,1331.0,1990.0,12.0,3.0,11207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,24275,李鑫,女,1994-11-03,未知,2011-01-01,\n,头顶上有一个红色的肉痔，大概有胡豆那么大，在我们家时叫何鑫，是我们家代养的，也称呼我们的父母...,（家长送养）在我们家时叫何鑫，是我们家代养的，也称呼我们的父母为爸爸、妈妈，我爸爸是开车的，...,2011/4/3 21:22:02,NaN,NaN,NaN,5903.0,2011.0,1.0,1.0,3873.0
52003,23443,吴文达,男,1988-09-08,155厘米左右,1999-02-10,澳门,"身材高瘦,单眼皮,大耳朵",NaN,2011/3/5 0:02:58,Macau,22.198745,113.543873,3807.0,1999.0,2.0,10.0,8216.0
52004,23233,寻找妹妹,女,1988-10-01,未知,1988-01-01,湖南省襄樊市,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,家长送养,2011/2/27 20:19:20,Xiangyang,32.006871,112.121945,-274.0,1988.0,1.0,1.0,12274.0
52005,22932,王梦楠,女,1994-01-24,145厘米左右,2001-12-28,天津市武进区,\n,（鹤姐跟进）在早晨上学的时候失踪，失踪时上小学1年级。,2011/2/21 13:56:55,Tianjin,39.341163,117.361784,2895.0,2001.0,12.0,28.0,7164.0


In [22]:
# prepping this for mapbox points map
# we're dropping rows that don't have coordinates
df.dropna(subset=['geo_lat'])
df.dropna(subset=['geo_lng'])

,ID,Name,Sex,DOB,Height_when_missing,Missing_date,Missing_location,Unique_traits,Other_info,Registration_datetime,geo_city,geo_lat,geo_lng,Missing_age(days),Year_missing,Month_missing,Day_missing,Days_missing
0,296933,卢雅琪,女,2002-12-29,165厘米左右,2017-09-22,"重庆市,市辖区,渝中区解放碑",上下牙龈有铁丝绵被。,NaN,2017/11/11 21:32:11,Chongqing,29.5572164,106.5771192,5381.0,2017.0,9.0,22.0,1417.0
1,199646,妹妹（遗弃）,女,1984-06-20,未知,1984-07-25,"重庆市,市辖区,解放碑附近",刚出生7天,NaN,2016/6/21 16:35:19,Chongqing,29.5572164,106.5771192,35.0,1984.0,7.0,25.0,13529.0
2,78228,黄雨锋,男,1991-12-11,80厘米左右,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,（燕子跟进）,2014/2/19 19:30:58,Shenzhen,22.53190021,114.2051181,1428.0,1995.0,11.0,8.0,9406.0
3,174869,林炜（送养）,男,1996-09-16,未知,1996-09-17,"北京市,市辖区,天坛",无法记清,NaN,2016/3/18 9:23:25,Beijing,39.8807799,116.4191967,1.0,1996.0,9.0,17.0,9092.0
4,144453,赵淑亮,女,1987-04-12,110厘米左右,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",大眼，短发额头有些小疤痕,（朵朵跟进）她那时说话不清楚,2015/9/2 7:05:07,Beijing,39.8713,116.3998,1331.0,1990.0,12.0,3.0,11207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52001,24489,不详（送养）,女,1972-12-20,未知,1973-01-01,天津市,不详,（家长送养）生母早亡，一岁时抱养到天津，当时养父在天津当炮兵，养父叫侯西珍（音），石家庄人；...,2011/4/8 18:13:06,Tianjin,39.341163,117.361784,12.0,1973.0,1.0,1.0,17752.0
52003,23443,吴文达,男,1988-09-08,155厘米左右,1999-02-10,澳门,"身材高瘦,单眼皮,大耳朵",NaN,2011/3/5 0:02:58,Macau,22.198745,113.543873,3807.0,1999.0,2.0,10.0,8216.0
52004,23233,寻找妹妹,女,1988-10-01,未知,1988-01-01,湖南省襄樊市,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,家长送养,2011/2/27 20:19:20,Xiangyang,32.006871,112.121945,-274.0,1988.0,1.0,1.0,12274.0
52005,22932,王梦楠,女,1994-01-24,145厘米左右,2001-12-28,天津市武进区,\n,（鹤姐跟进）在早晨上学的时候失踪，失踪时上小学1年级。,2011/2/21 13:56:55,Tianjin,39.341163,117.361784,2895.0,2001.0,12.0,28.0,7164.0


In [55]:
# 73 rows are dropped!
df.to_csv("pointsmapdata.csv")

In [83]:
df = pd.read_csv("combined_4am.csv")

/Users/isabellachua/.pyenv/versions/3.8.2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [86]:
df['Other_info']

0                                                      NaN
1                                                      NaN
2                                                   （燕子跟进）
3                                                      NaN
4                                           （朵朵跟进）她那时说话不清楚
                               ...                        
52002    （家长送养）在我们家时叫何鑫，是我们家代养的，也称呼我们的父母为爸爸、妈妈，我爸爸是开车的，...
52003                                                  NaN
52004                                                 家长送养
52005                          （鹤姐跟进）在早晨上学的时候失踪，失踪时上小学1年级。
52006    （漆彩生活跟进）离家出走，2010年11月21日早上6点从祁东县石门邮政所至育贤中学上学，中...
Name: Other_info, Length: 52007, dtype: object

In [87]:
df = df[[
    'ID', 
    'Name', 
    'Sex', 
    'DOB',
    'Missing_date',
    'Missing_location',
    'Days_missing',
    'Unique_traits', 
    'Other_info',
    'geo_city',
    'lat', 
    'long',
]]

In [88]:
df.dropna(subset=['lat'])
df.dropna(subset=['long'])

,ID,Name,Sex,DOB,Missing_date,Missing_location,Days_missing,Unique_traits,Other_info,geo_city,lat,long
0,296933,卢雅琪,女,2002-12-29,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417.0,上下牙龈有铁丝绵被。,NaN,Chongqing,29.5572164,106.5771192
1,199646,妹妹（遗弃）,女,1984-06-20,1984-07-25,"重庆市,市辖区,解放碑附近",13529.0,刚出生7天,NaN,Chongqing,29.5572164,106.5771192
2,78228,黄雨锋,男,1991-12-11,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406.0,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,（燕子跟进）,Shenzhen,22.53190021,114.2051181
3,174869,林炜（送养）,男,1996-09-16,1996-09-17,"北京市,市辖区,天坛",9092.0,无法记清,NaN,Beijing,39.8807799,116.4191967
4,144453,赵淑亮,女,1987-04-12,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207.0,大眼，短发额头有些小疤痕,（朵朵跟进）她那时说话不清楚,Beijing,39.8713,116.3998
...,...,...,...,...,...,...,...,...,...,...,...,...
52001,24489,不详（送养）,女,1972-12-20,1973-01-01,天津市,17752.0,不详,（家长送养）生母早亡，一岁时抱养到天津，当时养父在天津当炮兵，养父叫侯西珍（音），石家庄人；...,Tianjin,39.341163,117.361784
52003,23443,吴文达,男,1988-09-08,1999-02-10,澳门,8216.0,"身材高瘦,单眼皮,大耳朵",NaN,Macau,22.198745,113.543873
52004,23233,寻找妹妹,女,1988-10-01,1988-01-01,湖南省襄樊市,12274.0,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,家长送养,Xiangyang,32.006871,112.121945
52005,22932,王梦楠,女,1994-01-24,2001-12-28,天津市武进区,7164.0,\n,（鹤姐跟进）在早晨上学的时候失踪，失踪时上小学1年级。,Tianjin,39.341163,117.361784


In [11]:
df.keys()

Index(['ID', 'Name', 'Sex', 'DOB', 'Height_when_missing', 'Missing_date',
       'Missing_location', 'Unique_traits', 'Other_info',
       'Registration_datetime', 'geo_city', 'geo_lat', 'geo_lng',
       'Missing_age(days)', 'Year_missing', 'Month_missing', 'Day_missing',
       'Days_missing'],
      dtype='object')

In [19]:
df.to_csv("revised_geocoded12Aug.csv")

In [10]:
df1 = pd.read_csv("final_baobei_urls.csv")
df1.urls[0]

'https://m.baobeihuijia.com/PhotoShow/PhotoDetail.aspx?type=1&id=557216'

In [11]:
df1.head()

,Unnamed: 0,urls
0,0,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
1,1,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
2,2,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
3,3,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
4,4,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...


In [28]:
# extract id no, that ranges from 4-6 numbers
import re
url = df1['urls'] 

In [29]:
re.search(r'.*?\.gr/(.*?)(\d+)/$', url)

TypeError: expected string or bytes-like object

In [32]:
df1['ID'] = df1['urls'].str.extract(r"=([0-9]+)$") 

In [66]:
df1

,Unnamed: 0,urls,ID
0,0,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,296933
1,1,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,199646
2,2,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,78228
3,3,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,174869
4,4,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,144453
...,...,...,...
52250,52285,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,NaN
52251,52286,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,NaN
52252,52287,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,NaN
52253,52288,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...,NaN


In [5]:
df = pd.read_csv("revised_geocoded12Aug.csv")
df.head()

,Unnamed: 0,ID,Name,Sex,Missing_date,Missing_location,Days_missing,Unique_traits,geo_city,lat,long,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,0,296933,卢雅琪,女,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417,上下牙龈有铁丝绵被。,Chongqing,29.5572164,106.5771192,NaN,NaN,NaN,NaN,NaN
1,1,199646,妹妹（遗弃）,女,1984-07-25,"重庆市,市辖区,解放碑附近",13529,刚出生7天,Chongqing,29.5572164,106.5771192,NaN,NaN,NaN,NaN,NaN
2,2,78228,黄雨锋,男,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,Shenzhen,22.53190021,114.2051181,NaN,NaN,NaN,NaN,NaN
3,3,174869,林炜（送养）,男,1996-09-17,"北京市,市辖区,天坛",9092,无法记清,Beijing,39.8807799,116.4191967,NaN,NaN,NaN,NaN,NaN
4,4,144453,赵淑亮,女,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207,大眼，短发额头有些小疤痕,Beijing,39.8713,116.3998,NaN,NaN,NaN,NaN,NaN


In [6]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])

In [20]:
# shows rows where there's NaN values in any columns
df[df.isna().any(axis=1)]

,ID,Name,Sex,Missing_date,Missing_location,Days_missing,Unique_traits,geo_city,lat,long
707,43283,冯某(送养),女,NaN,贵州省遵义市红花岗区金鼎镇小溪村文星2队,NaN,幼时被领养,Zunyi,27.707064,106.795635
735,36298,无名(送养),女,NaN,贵州省遵义市习水县温水镇大水村,NaN,\n,Zunyi,28.45443,106.537413
741,35489,叶华生(送养),男,NaN,贵州省遵义市,NaN,1993年正月初8日出生，1993年正月初十送给一个四川人在遵义做百货的生意人抚养，现已18岁了。,Zunyi,27.725654,106.927389
1968,12950,朱伟杰,男,NaN,河南省周口市鹿邑县玄武镇,NaN,右脚脚趾曾断伤（打面机打断右脚四个脚趾）,Zhoukou,33.968059,115.27651
2133,35910,谭弟弟(送养),男,NaN,广东省中山市槎桥村槎桥踩石场,NaN,\n,Zhongshan,22.492549,113.3795
...,...,...,...,...,...,...,...,...,...,...
51987,34910,陈雯,女,1996-02-28,\n,9294,脚板底有一颗痣，肚子上有疤痕。,NaN,NaN,NaN
51989,33734,韩应扩,男,NaN,\n,NaN,他是我大爷，小的时候家穷送人，现在郑州，原第2毛纺厂上班，今年68岁，属猴，膝下可能有一女孩...,NaN,NaN,NaN
51997,29393,张小,女,NaN,\n,NaN,出生7天~,NaN,NaN,NaN
52000,26299,李依伦,男,2010-06-03,\n,4085,身穿短T恤尼色短裤,NaN,NaN,NaN


In [25]:
# removing rows without any lat/long values
df = df.dropna(subset=['lat', 'long'])

In [27]:
# 74 rows are removed
df.to_csv("29Aug_NaNremoved.csv")

In [89]:
# df_merged = df.merge(df1, left_on='ID', right_on='ID')
df1["ID"] = df["ID"].astype(str)
df["ID"] = df["ID"].astype(str)

<ipython-input-89-cb7a356a3079>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ID"] = df["ID"].astype(str)


In [90]:
df_merged = df.merge(df1, left_on='ID', right_on='ID')
df_merged

,ID,Name,Sex,DOB,Missing_date,Missing_location,Days_missing,Unique_traits,Other_info,geo_city,lat,long,Unnamed: 0,urls
0,296933,卢雅琪,女,2002-12-29,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417.0,上下牙龈有铁丝绵被。,NaN,Chongqing,29.5572164,106.5771192,0,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
1,199646,妹妹（遗弃）,女,1984-06-20,1984-07-25,"重庆市,市辖区,解放碑附近",13529.0,刚出生7天,NaN,Chongqing,29.5572164,106.5771192,1,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
2,78228,黄雨锋,男,1991-12-11,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406.0,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,（燕子跟进）,Shenzhen,22.53190021,114.2051181,2,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
3,174869,林炜（送养）,男,1996-09-16,1996-09-17,"北京市,市辖区,天坛",9092.0,无法记清,NaN,Beijing,39.8807799,116.4191967,3,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
4,144453,赵淑亮,女,1987-04-12,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207.0,大眼，短发额头有些小疤痕,（朵朵跟进）她那时说话不清楚,Beijing,39.8713,116.3998,4,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,24275,李鑫,女,1994-11-03,2011-01-01,\n,3873.0,头顶上有一个红色的肉痔，大概有胡豆那么大，在我们家时叫何鑫，是我们家代养的，也称呼我们的父母...,（家长送养）在我们家时叫何鑫，是我们家代养的，也称呼我们的父母为爸爸、妈妈，我爸爸是开车的，...,NaN,NaN,NaN,52037,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
52003,23443,吴文达,男,1988-09-08,1999-02-10,澳门,8216.0,"身材高瘦,单眼皮,大耳朵",NaN,Macau,22.198745,113.543873,52038,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
52004,23233,寻找妹妹,女,1988-10-01,1988-01-01,湖南省襄樊市,12274.0,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,家长送养,Xiangyang,32.006871,112.121945,52039,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
52005,22932,王梦楠,女,1994-01-24,2001-12-28,天津市武进区,7164.0,\n,（鹤姐跟进）在早晨上学的时候失踪，失踪时上小学1年级。,Tianjin,39.341163,117.361784,52040,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...


In [96]:
df_merged

,ID,Name,Sex,DOB,Missing_date,Missing_location,Days_missing,Unique_traits,Other_info,geo_city,lat,long,Unnamed: 0,urls
0,296933,卢雅琪,女,2002-12-29,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417.0,上下牙龈有铁丝绵被。,NaN,Chongqing,29.5572164,106.5771192,0,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
1,199646,妹妹（遗弃）,女,1984-06-20,1984-07-25,"重庆市,市辖区,解放碑附近",13529.0,刚出生7天,NaN,Chongqing,29.5572164,106.5771192,1,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
2,78228,黄雨锋,男,1991-12-11,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406.0,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,（燕子跟进）,Shenzhen,22.53190021,114.2051181,2,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
3,174869,林炜（送养）,男,1996-09-16,1996-09-17,"北京市,市辖区,天坛",9092.0,无法记清,NaN,Beijing,39.8807799,116.4191967,3,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
4,144453,赵淑亮,女,1987-04-12,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207.0,大眼，短发额头有些小疤痕,（朵朵跟进）她那时说话不清楚,Beijing,39.8713,116.3998,4,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,24275,李鑫,女,1994-11-03,2011-01-01,\n,3873.0,头顶上有一个红色的肉痔，大概有胡豆那么大，在我们家时叫何鑫，是我们家代养的，也称呼我们的父母...,（家长送养）在我们家时叫何鑫，是我们家代养的，也称呼我们的父母为爸爸、妈妈，我爸爸是开车的，...,NaN,NaN,NaN,52037,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
52003,23443,吴文达,男,1988-09-08,1999-02-10,澳门,8216.0,"身材高瘦,单眼皮,大耳朵",NaN,Macau,22.198745,113.543873,52038,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
52004,23233,寻找妹妹,女,1988-10-01,1988-01-01,湖南省襄樊市,12274.0,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,家长送养,Xiangyang,32.006871,112.121945,52039,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...
52005,22932,王梦楠,女,1994-01-24,2001-12-28,天津市武进区,7164.0,\n,（鹤姐跟进）在早晨上学的时候失踪，失踪时上小学1年级。,Tianjin,39.341163,117.361784,52040,https://m.baobeihuijia.com/PhotoShow/PhotoDeta...


In [97]:
# df_merged = df_merged.drop(columns=['Unnamed: 0_x'])
df_merged = df_merged.drop(columns=['Unnamed: 0'])

In [100]:
df = df_merged.sample(30000)

In [102]:
import numpy as np
df['Other_info'] = df['Other_info'].replace({
    np.nan : "无资料"
})

In [103]:
df['Other_info']

51434                                                    无
47523                                                    无
48610                                               （依依跟进）
26361                                                  无资料
47072                                 有他姨夫当年抱至常州戚墅堰火车站侯车室。
                               ...                        
24587                                                （回跟进）
51930    （云儿 跟进）当时会讲父母名字。出生日期：2003年07月1日（农历）失踪日期：2008年0...
1038                                                （洪姐跟进）
41466                                                  无资料
23969             当时大姑家住四川省南充县，南疆乡。后来南充划为了市，南疆乡也划入了高坪区，南疆乡
Name: Other_info, Length: 30000, dtype: object

In [104]:
df.to_csv("mergeddd_12Aug_30K.csv")

In [6]:
# finding gender breakdown of missing persons
df['Sex'].value_counts()

女     27680
男     24325
未知        2
Name: Sex, dtype: int64

In [7]:
df.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ID', 'Name', 'Sex', 'DOB',
       'Height_when_missing', 'Missing_date', 'Missing_location',
       'Unique_traits', 'Other_info', 'Registration_datetime', 'geo_city',
       'geo_lat', 'geo_lng', 'Missing_age(days)', 'Year_missing',
       'Month_missing', 'Day_missing', 'Days_missing'],
      dtype='object')

In [ ]:
df['Sex'].value_counts()

In [9]:
years = df['Year_missing'].value_counts()
years_missing = pd.DataFrame(years)
years_missing.to_csv("years_missing.csv")

### Round 3 — 28 Aug 2021

In [3]:
df = pd.read_csv("29Aug_NaNremoved.csv")

In [5]:
df.head()

,Unnamed: 0,ID,Name,Sex,Missing_date,Missing_location,Days_missing,Unique_traits,geo_city,lat,long
0,0,296933,卢雅琪,女,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417,上下牙龈有铁丝绵被。,Chongqing,29.5572164,106.5771192
1,1,199646,妹妹（遗弃）,女,1984-07-25,"重庆市,市辖区,解放碑附近",13529,刚出生7天,Chongqing,29.5572164,106.5771192
2,2,78228,黄雨锋,男,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,Shenzhen,22.53190021,114.2051181
3,3,174869,林炜（送养）,男,1996-09-17,"北京市,市辖区,天坛",9092,无法记清,Beijing,39.8807799,116.4191967
4,4,144453,赵淑亮,女,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207,大眼，短发额头有些小疤痕,Beijing,39.8713,116.3998


In [6]:
df = df.drop(columns=['Unnamed: 0'])

In [7]:
df

,ID,Name,Sex,Missing_date,Missing_location,Days_missing,Unique_traits,geo_city,lat,long
0,296933,卢雅琪,女,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417,上下牙龈有铁丝绵被。,Chongqing,29.5572164,106.5771192
1,199646,妹妹（遗弃）,女,1984-07-25,"重庆市,市辖区,解放碑附近",13529,刚出生7天,Chongqing,29.5572164,106.5771192
2,78228,黄雨锋,男,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,Shenzhen,22.53190021,114.2051181
3,174869,林炜（送养）,男,1996-09-17,"北京市,市辖区,天坛",9092,无法记清,Beijing,39.8807799,116.4191967
4,144453,赵淑亮,女,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207,大眼，短发额头有些小疤痕,Beijing,39.8713,116.3998
...,...,...,...,...,...,...,...,...,...,...
51928,24489,不详（送养）,女,1973-01-01,天津市,17752,不详,Tianjin,39.341163,117.361784
51929,23443,吴文达,男,1999-02-10,澳门,8216,"身材高瘦,单眼皮,大耳朵",Macau,22.198745,113.543873
51930,23233,寻找妹妹,女,1988-01-01,湖南省襄樊市,12274,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,Xiangyang,32.006871,112.121945
51931,22932,王梦楠,女,2001-12-28,天津市武进区,7164,\n,Tianjin,39.341163,117.361784


In [8]:
df[df.isna().any(axis=1)]

,ID,Name,Sex,Missing_date,Missing_location,Days_missing,Unique_traits,geo_city,lat,long
707,43283,冯某(送养),女,NaN,贵州省遵义市红花岗区金鼎镇小溪村文星2队,NaN,幼时被领养,Zunyi,27.707064,106.795635
735,36298,无名(送养),女,NaN,贵州省遵义市习水县温水镇大水村,NaN,\n,Zunyi,28.45443,106.537413
741,35489,叶华生(送养),男,NaN,贵州省遵义市,NaN,1993年正月初8日出生，1993年正月初十送给一个四川人在遵义做百货的生意人抚养，现已18岁了。,Zunyi,27.725654,106.927389
1968,12950,朱伟杰,男,NaN,河南省周口市鹿邑县玄武镇,NaN,右脚脚趾曾断伤（打面机打断右脚四个脚趾）,Zhoukou,33.968059,115.27651
2133,35910,谭弟弟(送养),男,NaN,广东省中山市槎桥村槎桥踩石场,NaN,\n,Zhongshan,22.492549,113.3795
...,...,...,...,...,...,...,...,...,...,...
48602,42151,无,女,NaN,贵州省毕节市大方县双山镇高坪村,NaN,被拐卖时还是小孩，和有轻度智障母亲一起被卖的，所以有什么特征我不太清楚。脸有点圆。,Bijie,27.202786,105.506629
48902,27425,陈周,男,NaN,安徽省五河县中兴路,NaN,清瘦 嗓子沙哑,Bengbu,33.143083,117.8909852
49443,31888,符志扬,男,NaN,北京天安门广场东面火车站,NaN,带有儋州口音 ， 从小生活在海南省儋州市那大镇洛基番真村委会番真村，走失时14岁,Beijing,39.9054895,116.3976317
49444,31887,符志云,男,NaN,北京天安门广场东面火车站,NaN,带有儋州口音 ， 从小生活在海南省儋州市那大镇洛基番真村委会番真村 ，走失时12岁。,Beijing,39.9054895,116.3976317


In [11]:
df = df.fillna("无")

In [14]:
df.to_csv("new.csv", index=False)

In [16]:
df.Unique_traits.replace({
    ' ': '无',
    '\n': '无'
})

0                                               上下牙龈有铁丝绵被。
1                                                    刚出生7天
2                                 失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。
3                                                     无法记清
4                                             大眼，短发额头有些小疤痕
                               ...                        
51928                                                   不详
51929                                         身材高瘦,单眼皮,大耳朵
51930    妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...
51931                                                    无
51932                    身穿乔丹牌天蓝色外套，浅蓝色牛仔裤，白色球鞋。身高173CM左右。
Name: Unique_traits, Length: 51933, dtype: object

In [19]:
df.Unique_traits == '\n'

0        False
1        False
2        False
3        False
4        False
         ...  
51928    False
51929    False
51930    False
51931     True
51932    False
Name: Unique_traits, Length: 51933, dtype: bool

In [20]:
df

,ID,Name,Sex,Missing_date,Missing_location,Days_missing,Unique_traits,geo_city,lat,long
0,296933,卢雅琪,女,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417,上下牙龈有铁丝绵被。,Chongqing,29.5572164,106.5771192
1,199646,妹妹（遗弃）,女,1984-07-25,"重庆市,市辖区,解放碑附近",13529,刚出生7天,Chongqing,29.5572164,106.5771192
2,78228,黄雨锋,男,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,Shenzhen,22.53190021,114.2051181
3,174869,林炜（送养）,男,1996-09-17,"北京市,市辖区,天坛",9092,无法记清,Beijing,39.8807799,116.4191967
4,144453,赵淑亮,女,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207,大眼，短发额头有些小疤痕,Beijing,39.8713,116.3998
...,...,...,...,...,...,...,...,...,...,...
51928,24489,不详（送养）,女,1973-01-01,天津市,17752,不详,Tianjin,39.341163,117.361784
51929,23443,吴文达,男,1999-02-10,澳门,8216,"身材高瘦,单眼皮,大耳朵",Macau,22.198745,113.543873
51930,23233,寻找妹妹,女,1988-01-01,湖南省襄樊市,12274,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,Xiangyang,32.006871,112.121945
51931,22932,王梦楠,女,2001-12-28,天津市武进区,7164,\n,Tianjin,39.341163,117.361784


In [26]:
cols = ['ID', 'DOB']
df1 = pd.read_csv("ready_for_viz.csv", skipinitialspace=True, usecols=cols)

In [28]:
df_combined = df.merge(df1, left_on = "ID", right_on= "ID")

In [29]:
df_combined

,ID,Name,Sex,Missing_date,Missing_location,Days_missing,Unique_traits,geo_city,lat,long,DOB
0,296933,卢雅琪,女,2017-09-22,"重庆市,市辖区,渝中区解放碑",1417,上下牙龈有铁丝绵被。,Chongqing,29.5572164,106.5771192,2002-12-29
1,199646,妹妹（遗弃）,女,1984-07-25,"重庆市,市辖区,解放碑附近",13529,刚出生7天,Chongqing,29.5572164,106.5771192,1984-06-20
2,78228,黄雨锋,男,1995-11-08,"广东省,深圳市,沙头角暗径田东夜市",9406,失踪时身穿紫红色，圆领长衫，灰白色背心套，赤脚。,Shenzhen,22.53190021,114.2051181,1991-12-11
3,174869,林炜（送养）,男,1996-09-17,"北京市,市辖区,天坛",9092,无法记清,Beijing,39.8807799,116.4191967,1996-09-16
4,144453,赵淑亮,女,1990-12-03,"北京市,市辖区,当时的永定门车站侯车室",11207,大眼，短发额头有些小疤痕,Beijing,39.8713,116.3998,1987-04-12
...,...,...,...,...,...,...,...,...,...,...,...
51928,24489,不详（送养）,女,1973-01-01,天津市,17752,不详,Tianjin,39.341163,117.361784,1972-12-20
51929,23443,吴文达,男,1999-02-10,澳门,8216,"身材高瘦,单眼皮,大耳朵",Macau,22.198745,113.543873,1988-09-08
51930,23233,寻找妹妹,女,1988-01-01,湖南省襄樊市,12274,妹妹是出生10天经过一个中间人介绍抱养给襄樊三医院附近一个王姓人家，抱养男的在三医院的附近的...,Xiangyang,32.006871,112.121945,1988-10-01
51931,22932,王梦楠,女,2001-12-28,天津市武进区,7164,\n,Tianjin,39.341163,117.361784,1994-01-24


In [30]:
df_combined.to_csv("28Aug_third.csv", index=False)

In [31]:
cols = ['ID', 'Year_missing']
df2 = pd.read_csv("combined_4am.csv", skipinitialspace=True, usecols=cols)

In [32]:
df2

,ID,Year_missing
0,296933,2017
1,199646,1984
2,78228,1995
3,174869,1996
4,144453,1990
...,...,...
52002,24275,2011
52003,23443,1999
52004,23233,1988
52005,22932,2001


In [33]:
df_combined2 = df_combined.merge(df2, left_on = "ID", right_on= "ID")

In [37]:
df_combined2.to_csv("28Aug_fourth.csv", index=False)